In [1]:
!pip install pillow opencv-python
!sudo apt-get install tesseract-ocr
!pip install pytesseract

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 45 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 2s (2,672 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debc

In [2]:
from PIL import Image
import pytesseract
import cv2 as cv
import numpy as np

In [ ]:
!tesseract --version


tesseract 4.1.1
 leptonica-1.82.0
  libgif 5.1.9 : libjpeg 8d (libjpeg-turbo 2.1.1) : libpng 1.6.37 : libtiff 4.3.0 : zlib 1.2.11 : libwebp 1.2.2 : libopenjp2 2.4.0
 Found AVX2
 Found AVX
 Found FMA
 Found SSE
 Found libarchive 3.6.0 zlib/1.2.11 liblzma/5.2.5 bz2lib/1.0.8 liblz4/1.9.3 libzstd/1.4.8


In [ ]:
path = "/content/drive/MyDrive/Colab Notebooks/page_01.jpg"


In [ ]:
im = Image.open(path)
im.size
np_img = np.array(im)


# Inverted images

cv takes np array so converting the Image to np array and then back to Image for displaying


In [ ]:
inv_img = cv.bitwise_not(np_img)
inv_img = Image.fromarray(inv_img)
# inv_img



# Binarization and greyscaling

In [ ]:
def grayscale(image):
  return cv.cvtColor(image,cv.COLOR_BGR2GRAY)

In [ ]:
gray_image_np = grayscale(np_img)
gray_image = Image.fromarray(gray_image_np)
# gray_image

In [ ]:
tresh, im_bw_np= cv.threshold(gray_image_np, 210, 230, cv.THRESH_BINARY)
im_bw = Image.fromarray(im_bw_np)


# Noise removal

# Refer this - https://github.com/wjbmattingly/ocr_python_textbook/blob/main/02_02_working%20with%20opencv.ipynb

In [ ]:
# reading a pdf file and then converting it into images to perform OCR


In [ ]:
!apt-get install -y poppler-utils


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
poppler-utils is already the newest version (22.02.0-2ubuntu0.4).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [ ]:
!pip install pdf2image

In [ ]:
path = "/content/drive/MyDrive/Colab Notebooks/Preprint.pdf"

In [ ]:
from pdf2image import convert_from_path
import os

# Store Pdf with convert_from_path function
images = convert_from_path(path)

if not os.path.isfile(path):
    print(f"Error: The file '{path}' does not exist.")
else:
    # Store Pdf with convert_from_path function
    images = convert_from_path(path)

    # Create a directory if it doesn't exist
    output_dir = "/content/drive/MyDrive/Colab Notebooks/OCRImages/"
    os.makedirs(output_dir, exist_ok=True)

    for i, image in enumerate(images):
        # Save pages as images in the pdf
        image_path = os.path.join(output_dir, f"page_{i+1}.jpg")
        image.save(image_path, 'JPEG')
        print(f"Page {i+1} saved as {image_path}")

    print("Conversion complete.")

Page 1 saved as /content/drive/MyDrive/Colab Notebooks/OCRImages/page_1.jpg
Page 2 saved as /content/drive/MyDrive/Colab Notebooks/OCRImages/page_2.jpg
Page 3 saved as /content/drive/MyDrive/Colab Notebooks/OCRImages/page_3.jpg
Page 4 saved as /content/drive/MyDrive/Colab Notebooks/OCRImages/page_4.jpg
Page 5 saved as /content/drive/MyDrive/Colab Notebooks/OCRImages/page_5.jpg
Page 6 saved as /content/drive/MyDrive/Colab Notebooks/OCRImages/page_6.jpg
Page 7 saved as /content/drive/MyDrive/Colab Notebooks/OCRImages/page_7.jpg
Page 8 saved as /content/drive/MyDrive/Colab Notebooks/OCRImages/page_8.jpg
Page 9 saved as /content/drive/MyDrive/Colab Notebooks/OCRImages/page_9.jpg
Page 10 saved as /content/drive/MyDrive/Colab Notebooks/OCRImages/page_10.jpg
Conversion complete.


In [ ]:
# removing some unuseful images
os.remove("/content/drive/MyDrive/Colab Notebooks/OCRImages/page_1.jpg")
os.remove("/content/drive/MyDrive/Colab Notebooks/OCRImages/page_9.jpg")
os.remove("/content/drive/MyDrive/Colab Notebooks/OCRImages/page_10.jpg")

In [ ]:
# some preprocessing of the image -
import pytesseract
from PIL import Image
import cv2 as cv
import numpy as np

In [ ]:
image1 = cv.imread("/content/drive/MyDrive/Colab Notebooks/OCRImages/page_2.jpg")
image1_np = np.array(image1)

In [ ]:
gray_np = cv.cvtColor(image1, cv.COLOR_BGR2GRAY)
gray_image = Image.fromarray(gray_np)
# gray_image

In [ ]:
# blurring the image -
blur_np = cv.GaussianBlur(gray_np, (7, 7), 3)
blur_image = Image.fromarray(blur_np)
# blur_image

In [ ]:
thresh = cv.threshold(blur_np, 0, 255, cv.THRESH_BINARY + cv.THRESH_OTSU)[1]

In [ ]:
thresh_image = Image.fromarray(thresh)
# thresh_image

In [ ]:
kernel = cv.getStructuringElement(cv.MORPH_RECT, (3, 13))
dilated = cv.dilate(thresh, kernel, iterations=1)

In [ ]:
dilated_image = Image.fromarray(dilated)
# dilated_image

# Finding the contours

In [ ]:
cnts = cv.findContours(dilated, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
cnts = cnts[0] if len(cnts) == 2 else cnts[1]

In [ ]:
cnts = sorted(cnts, key = lambda x: cv.boundingRect(x)[0])

In [ ]:
for c in cnts:
  x,y,w,h = cv.boundingRect(c)
  if h>200 and w>20:
    cv.rectangle(image1_np, (x, y), (x + w, y + h), (36, 255, 12), 2)
  cv.imwrite("/content/drive/MyDrive/Colab Notebooks/OCRImages/page_2_bbox.jpg", image1_np)


# Preprocessing
below are the steps for preprocessing

In [ ]:
import cv2 as cv
import numpy as np
from PIL import Image

image_path = "/content/drive/MyDrive/Colab Notebooks/OCRImages/page_2.jpg"
im = Image.open(image_path)
image1_np = np.array(im)

gray = cv.cvtColor(image1_np, cv.COLOR_BGR2GRAY)

blur = cv.GaussianBlur(gray, (5, 5), 0)

_, thresh = cv.threshold(blur, 0, 255, cv.THRESH_BINARY_INV + cv.THRESH_OTSU)

kernel = cv.getStructuringElement(cv.MORPH_RECT, (5, 5))
dilated = cv.dilate(thresh, kernel, iterations=1)

height, width = dilated.shape
left_half = dilated[:, :width//2]
right_half = dilated[:, width//2:]

def draw_bounding_boxes(image, dilated_part, x_offset=0):
    cnts, _ = cv.findContours(dilated_part, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    for c in cnts:
        x, y, w, h = cv.boundingRect(c)
        if h > 20 and w > 20:
            cv.rectangle(image, (x + x_offset, y), (x + w + x_offset, y + h), (36, 255, 12), 2)

draw_bounding_boxes(image1_np, left_half)
draw_bounding_boxes(image1_np, right_half, x_offset=width//2)

cv.imwrite("/content/drive/MyDrive/Colab Notebooks/OCRImages/page_2_bbox.jpg", image1_np)


True

In [8]:
imgfile = "/content/drive/MyDrive/Colab Notebooks/OCRImages/page_2.jpg"
img = Image.open(imgfile)

In [9]:
# sample image 2
# imgfile = "/content/drive/MyDrive/Colab Notebooks/OCRImages/deletenow.png"
# img = Image.open(imgfile)

In [10]:
ocr_result = pytesseract.image_to_string(img)
print(ocr_result)

A Review of Generative AI from Historical
Perspectives

Dipankar Dasgupta
Department of Computer Science
University of Memphis
Memphis, USA
dasgupta@memphis.edu

Abstract—Many applications of Generative AI (such as DALL-
E, GPT-3, ChatGPT, etc.) are making headline news in recent
months and have been receiving both praise and criticism for
their far reaching implications. Some of these applications include
query responses, language translation, text to images and videos,
composing stories, essays, creating arts and music, generating
programs, etc. This review provides an historical background of
Generative AI techniques and how they evolved over the years.
This report highlights the benefits of Generative AI technologies
and their limitations/challenges in moving forward. It is also to be
noted that the large-scale applications of AI and their successes
are now possible due to exponential advances in hardware
(computational power, storage capacity), cloud computing and
related operatio

In [11]:
import re

def remove_author_info(text):

    authors = [
        {
            "name": "Dipankar Dasgupta",
            "details": [
                "Department of Computer Science",
                "University of Memphis",
                "Memphis, USA",
                "dasgupta@memphis.edu"
            ]
        },
        {
            "name": "Deepak Venugopal",
            "details": [
                "Department of Computer Science",
                "University of Memphis",
                "Memphis, USA",
                "dvenugopal@memphis.edu"
            ]
        },
        {
            "name": "Kishor Datta Gupta",
            "details": [
                "Department of Computer Science",
                "Clark Atlanta University",
                "Atlanta, USA",
                "kgupta@cau.edu"
            ]
        }
    ]
    cleaned_text = text
    for author in authors:
        cleaned_text = re.sub(re.escape(author["name"]) + r'.*?\n', '', cleaned_text, flags=re.DOTALL)
        for detail in author["details"]:
            cleaned_text = re.sub(re.escape(detail), '', cleaned_text)
    cleaned_text = re.sub(r'\n\s*\n', '\n', cleaned_text)

    return cleaned_text.strip()

In [12]:
text_cleaned = remove_author_info(ocr_result)
text_cleaned

'A Review of Generative AI from Historical\nPerspectives\nAbstract—Many applications of Generative AI (such as DALL-\nE, GPT-3, ChatGPT, etc.) are making headline news in recent\nmonths and have been receiving both praise and criticism for\ntheir far reaching implications. Some of these applications include\nquery responses, language translation, text to images and videos,\ncomposing stories, essays, creating arts and music, generating\nprograms, etc. This review provides an historical background of\nGenerative AI techniques and how they evolved over the years.\nThis report highlights the benefits of Generative AI technologies\nand their limitations/challenges in moving forward. It is also to be\nnoted that the large-scale applications of AI and their successes\nare now possible due to exponential advances in hardware\n(computational power, storage capacity), cloud computing and\nrelated operational layers of software.\nIndex Terms—Machine Learning, Deep Neural Network, Evo-\nlutionary

In [ ]:
context_text = ""
for i in range(2,9):
  imgfile = "/content/drive/MyDrive/Colab Notebooks/OCRImages/page_"+str(i)+".jpg"
  img = Image.open(imgfile)
  ocr_result = pytesseract.image_to_string(img)
  text_cleaned = remove_author_info(ocr_result)
  context_text += text_cleaned

# print(context_text)
len(context_text)

37652

In [ ]:
context_text[:100]

'A Review of Generative AI from Historical\nPerspectives\nAbstract—Many applications of Generative AI ('

In [ ]:
# now the above text can be used as an context for the RAG application.


In [ ]:
file = open("/content/drive/MyDrive/Colab Notebooks/OCRImages/Contextdoc.txt", "w")
file.write(context_text)
file.close()